# Introduction
This practical work is about creation of elementary models in keras.
While it focuses on image classification, it illustrates many important functionalities of the keras framework and tries to provide the user with good deep-learning development strategies.

## Setup
Before you start, please mount your google drive.
For each exercise, create an output directory to store monitoring data and your model.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from tensorflow import keras

from tensorflow.keras import layers
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

from keras.datasets import mnist, fashion_mnist, cifar10
from keras import metrics, utils
from keras.utils import to_categorical


import numpy as np
from numpy import expand_dims

In [0]:
import os
save_dir = os.path.join(os.getcwd(), 'saved_models')

# EX1: Lenet-like classifier.

## 1- Build model architecture
The architecture has to be a CLASS called "Lenet_like".

- I can define the width: number of filters in the first layer.
- I can define the depth: number of conv layers.
- I can specify the number of classes: output neurons.

Lenet_like has no input tensor. But it has a \_\_call\_\_ function and can therefore be called on an input later.

The rule to go from depth d to depth d+1, is to reduce the spatial size by a factor of 2 in each direction.

Hidden dense layer will have 512 units.

In [0]:
class Lenet_like:
    # Initialisation
  def _init_(self, width, depth, output):
    self.width = width            # number of filters in the first layer 
    self.depth = depth            # number of conv layers
    self.output = output          # number of classes

  def  __call__(self, x_):
      # input layer: relu activation function
    y_ = Conv2D(filters = 32, padding = 'same', activation = 'relu')(x_) 
    
      # hidden layers: reducion spatial size by 2
    y_ = (Conv2D(32, (3, 3), padding = "same", activation = "relu"))(y_)
    y_ = (MaxPooling2D(pool_size=(2, 2)))(y_)
    
    y_ = (Conv2D(64, (3, 3), padding = "same", activation = "relu"))(y_)
    y_ = (MaxPooling2D(pool_size=(2, 2)))(y_)
   
    y_ = (Conv2D(128, (3, 3), padding = "same", activation = "relu"))(y_)
    y_ = (MaxPooling2D(pool_size=(2, 2)))(y_)
   
    y_ = (Conv2D(256, (3, 3), padding = "same", activation = "relu"))(y_)
    y_ = (MaxPooling2D(pool_size = (2, 2)))(y_)
    
    y_ = (Conv2D(512, (3, 3), padding = "same", activation = "relu"))(y_)
    y_ = (MaxPooling2D(pool_size = (2, 2)))(y_)
    
      # output layer: softmax activation function instead of relu
    y_=(Dense(512, pandding = 'same', activation = 'softmax'))    
    
    return y_


## 2- A function to create a model with Lenet_like architecture
I want the model to be able to fit on the following datasets:

- mnist
- fashion-mnist[texte du lien](https://)
- cifar-10

Create a function called "make_lenet_model" that take the name of one of these dataset as a str.
It returns a keras Model object.
It should obviously take all arguments to init the Lenet_like architecture.
Arguments other than the dataset might have default values.
I want to be able to monitor the accuracy of the model.

### About the datasets:

#### MNIST:  
- images of handwritten numbers
- 70 000 images (60 000/10 000)
- size 28x28 (width = height = 28)
- grayscale (depth = 1)
- 10 classes (0 to 9)

#### FASHION-MNIST
- images of clothes
- 70 000 images (60 000/10 000)
- size 28x28
- grayscale (depth = 1)
- 10 classes (ex:T-shirt, trousers, dress...)

#### CIFAR-10
- photos of objects
- 60 000 images (50 000/10 000)
- size 32x32
- color (depth=3)
- 10 classes (ex: airplanes, automobile, bird, cat...)


In [0]:
def make_lenet_model(dataset, output=10):
  (x_train, y_train), (x_test, y_test) = dataset.load_data()

  if (dataset == "mnist" or dataset == "fashion_mnist"):
    width = 28       
    depth = 1

  elif (dataset == "cifar10"):
    width = 32   
    depth=3
      # reshape to 28x28 and to grayscale
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

   # convert from integers to floats
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')

    # normalize to range 0-1
  x_train = x_train / 255.0
  x_test = x_test / 255.0

    # one hot encode target values
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)
    
  x_ = keras.Input(x_train.shape[-3:], name='input_height_width')  # remove depth
  y_ = Lenet_like(width, depth, nb_classes)(x_)

    # create model
  model = keras.Model(inputs = x_, outputs = y_)

    # compile model to monitor the accuracy of the model
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # fit model
  model.fit(x_train, y_train, epochs=12, batch_size=128, verbose=1, validation_data=(x_test, y_test))

    # evaluate the model
  scores = model.evaluate(x_test, y_test, verbose=1)  # verbose: 0 = silent, 1 = progress bar, 2 = one line per epoch.

  print('Test loss:', scores[0])
  print('Test accuracy:', scores[1])

  model_name = "keras"+dataset
  if not os.path.isdir(save_dir):
      os.makedirs(save_dir)
  model_path = os.path.join(save_dir, model_name)
  model.save(model_path)

  return model, (x_train, x_test), (y_train, y_test)


## 3- Create a fitting function

I want a function "fit_model_on" with the following arguments:

- dataset: str name of the dataset
- epochs: number of times you fit on the entire training set
- batch_size: number of images to average gradient on

The function must create a Lenet model and fit it following these parameters.
The function should:

- fit the model, obviously
- store the model architecture in a .json file in your output directory
- store the model's weights in a .h5 file in your output directory
- store the fitting metrics loss, validation_loss, accuracy, validation_accuracy under the form of a plot exported in a png file.

Run your fitting function of course.

In [0]:
def fit_model_on(dataset, depth=1, epochs=10, batch_size=128):
    model, (x_train, x_test), (y_train, y_test) = make_lenet_model(dataset, output=10)
    
    model.fit(x_train, y_train,
              batch_size = batch_size,
              epochs = epochs,
              verbose = 1,
              validation_data = (x_test, y_test))

In [32]:
fit_model_on(mnist, depth=1, epochs=10, batch_size=128)

UnboundLocalError: ignored